importing libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
#nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import re
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

Loading dataset

In [2]:
dataset=pd.read_csv('hate_speech_detection .csv')
tweets=dataset.iloc[:,2].values
test=[dataset.iloc[1,2]] #for checking the working of clean 
y=dataset.drop(['tweet','id'],axis='columns')
y.value_counts()

label
0        3000
1        2242
dtype: int64

Cleaning text

In [3]:
def clean(text):
  STOPWORDS.update({'is','the','of','to','on','so'})
  clean_tweet=[]
  for i in range(len(text)):
    sample=re.sub(r'[^a-zA-Z\n]'," ",text[i])
    sample=re.sub(r"[.,'\"@#!?]"," ",sample)
    sample=re.sub('user'," ",sample)
    sample=sample.lower()
    sample=sample.split()
    lm=WordNetLemmatizer()
    for word in sample:
      word=lm.lemmatize(word)
    sample=list(set(sample)-STOPWORDS)
    sample=" ".join(sample)
    clean_tweet.append(sample) 
  return clean_tweet
type(clean(tweets))#Showing the 10 clean tweets from dataset 

list

In [4]:
def get_predicton(tweet):
  lis=[[]]
  lis[0].append(tweet)
  tweet=lis
  tweet=clean(tweet[0])
  lol=cv.transform(clean(tweet))
  pred=rf.predict(lol)
  if pred[0]==1:
    result='Toxic tweet'
  else:
    result='Non-Toxic tweet'

  return result

Creating tokens

In [5]:
cv=CountVectorizer(max_features=5000)
x=cv.fit_transform(clean(tweets)).toarray()

Tackling imbalence in dataset

In [6]:
sm=SMOTE()
x_sm,y_sm=sm.fit_resample(x,y)

In [7]:
y_sm=y_sm['label'].values #making it nd array to feed the model

Splitting dataset in to test and training set

In [8]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x_sm,y_sm,test_size=0.3,random_state=0)
type(ytrain)

numpy.ndarray

Training model on training set

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,criterion='gini',max_features='log2') #best parameter from Grid search is used in this model
rf.fit(xtrain,ytrain)
ypred=rf.predict(xtest)

Grid Search with cross validation

In [10]:
param=[{
    'n_estimators':[50,100,120],
    'criterion':['gini','entropy'],
    'max_features':['sqrt','log2','']}]
gw=GridSearchCV(estimator=rf,param_grid=param,scoring='accuracy',cv=5)
gw.fit(xtrain,ytrain)
best_acc=gw.best_score_
beat_param=gw.best_params_
print('Best Accuracy: {:.2f} %'.format(best_acc*100) )
print('best param:',beat_param )


"param=[{\n    'n_estimators':[50,100,120],\n    'criterion':['gini','entropy'],\n    'max_features':['sqrt','log2','']}]\ngw=GridSearchCV(estimator=rf,param_grid=param,scoring='accuracy',cv=5)\ngw.fit(xtrain,ytrain)\nbest_acc=gw.best_score_\nbeat_param=gw.best_params_\nprint('Best Accuracy: {:.2f} %'.format(best_acc*100) )\nprint('best param:',beat_param )\n"

Checking accuracy

In [11]:
from sklearn.metrics import classification_report
print('Classification Report of the Random Forest model: \n',classification_report(ytest,ypred))

Classification Report of the Random Forest model: 
               precision    recall  f1-score   support

           0       0.89      0.82      0.85       895
           1       0.83      0.90      0.86       905

    accuracy                           0.86      1800
   macro avg       0.86      0.86      0.86      1800
weighted avg       0.86      0.86      0.86      1800



Testing model on a random tweet

In [12]:
print(get_predicton("@user you never answered me about your statement on the #nazi's over #hatred, &amp; #bigotry being found wâ¦"))

['statement bigotry amp hatred answered nazi found s never w']
Toxic tweet
